In [5]:
import requests
from geopy.distance import geodesic

# Replace with your actual Google Maps API key
GOOGLE_MAPS_API_KEY = 'YOUR_NEW_GOOGLE_MAPS_API_KEY'

def get_route(origin, destination):
    """Get the route from origin to destination using Google Maps API."""
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    route = response.json()

    # Check if the route contains any routes
    if 'routes' not in route or len(route['routes']) == 0:
        print("Error: No routes found.")
        print("Response from API:", route)
        return None

    return route

def get_charging_stations(location, radius=50000):
    """Get EV charging stations near a given location using Google Maps Places API."""
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type=charging_station&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    stations = response.json().get('results', [])
    return stations

def calculate_distance(origin, destination):
    """Calculate driving distance between two points using Google Maps API."""
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url)
    directions = response.json()

    if 'routes' in directions and len(directions['routes']) > 0:
        distance = directions['routes'][0]['legs'][0]['distance']['value']  # in meters
        return distance / 1000  # convert to kilometers
    else:
        print("Error: Could not calculate distance.")
        return None

def find_nearest_station(origin, destination, max_range, user_filters=None):
    """Find the nearest charging station within range and calculate distance to destination."""
    stations = get_charging_stations(origin, radius=max_range*1000)

    if user_filters:
        # Apply user-defined filters (e.g., station type, available chargers)
        stations = [station for station in stations if user_filters(station)]

    closest_station = None
    shortest_distance_to_destination = float('inf')

    for station in stations:
        station_location = f"{station['geometry']['location']['lat']},{station['geometry']['location']['lng']}"
        distance_to_destination = calculate_distance(station_location, destination)

        if distance_to_destination is not None and distance_to_destination < shortest_distance_to_destination:
            shortest_distance_to_destination = distance_to_destination
            closest_station = station

    if closest_station:
        return closest_station, shortest_distance_to_destination
    else:
        print("No suitable charging stations found.")
        return None, None

# Example usage
origin = "37.7749,-122.4194"  # San Francisco, CA
destination = "34.0522,-118.2437"  # Los Angeles, CA
max_range = 400  # km

closest_station, distance_to_destination = find_nearest_station(origin, destination, max_range)
if closest_station:
    print(f"Closest charging station: {closest_station['name']}")
    print(f"Distance to destination from station: {distance_to_destination} km")
else:
    print("No suitable charging stations found.")


No suitable charging stations found.
No suitable charging stations found.
